### Setup

In [1]:
# load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from scipy.stats import norm, t
from arch import arch_model
from time import perf_counter

In [2]:
# classes and functions from 'util'
from util.data_load import returns_load
from util.ArchModelWrapper import ArchModelWrapper
from util.DescStat import DescStat
from util.Standardizer import Standardizer
from util.TrainTestSplitter import Splitter
from util.ANN_core import NeuralNetwork

In [3]:
# setting seed for reproducibility
np.random.seed(1995)

In [57]:
# Variables used in code
start_dt = '2000-01-01'
end_dt = '2020-03-01'
ticker = 'MSFT' 
interval = '1d'

In [58]:
ret_data = returns_load(ticker=ticker,
                    interval=interval,
                    start_dt=start_dt,
                    end_dt=end_dt)
ret_data.set_index('dt', inplace=True)

[*********************100%***********************]  1 of 1 completed


#### Demeaned Returns
 this can be skipped if want to estiamte mu (not a problem in arch, however problematic in custom ANN)

In [60]:
ret_data['return100'] = ret_data['return100'] - np.mean(ret_data['return100'])

### Data Split

In [61]:
Splitter_all = Splitter(train_part=0.7)
Splitter_garch = Splitter(train_part=0.1)
[train_data, test_data] = Splitter_all.split(dataset = ret_data['return100'])
[est_data_garch, eval_data_garch] = Splitter_garch.split(dataset = train_data)

Last observation in trainset: 2014-02-12 00:00:00 , 
            first observation in testset: 2014-02-13 00:00:00
Last observation in trainset: 2001-05-30 00:00:00 , 
            first observation in testset: 2001-05-31 00:00:00


In [62]:
test_data.shape

(1521, 1)

### Garch estimation

In [63]:
models = ['GARCH', 'EGARCH']
p = list(range(1,3))
o = list(range(1,3))
q = list(range(1,3))
dist = ['Normal', 'StudentsT']
index = pd.MultiIndex.from_product([models, p, o, q, dist], names = ['models', 'p', 'o', 'q', 'dist'])
space = pd.DataFrame(index = index).reset_index()
space = space[~((space['models']=='GARCH') & (space['o']==1))]
space.loc[((space['models']=='GARCH') & (space['o']==2)),'o'] = 0

In [64]:
ArchMod = ArchModelWrapper(train_data=train_data, eval_data_garch=eval_data_garch)

In [65]:
time_begin = perf_counter()
space['llik'] = space.apply(lambda row: ArchMod.estimate_predict(row['models'], row['p'], row['o'], row['q'], row['dist']),
                                    axis=1)
f'Time of calculations: {round((perf_counter()-time_begin)/60, 0)} min'

C:\Users\sskor\Anaconda3\lib\site-packages\arch\univariate\base.py:709: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,


KeyboardInterrupt: 

In [12]:
predictions = ArchMod.get_predictions() 

### Select best specifications

In [13]:
best_spec = (
    space
        .groupby(['models', 'dist'])
        .agg({'llik':'max'})[['llik']]
        .reset_index()
        .merge(space, on=['models','dist','llik'])
)

SpecificationError: Column(s) ['llik'] do not exist

In [17]:
best_spec_strings = best_spec.apply(lambda row : f'{row["models"]}-{row["dist"]}-{row["p"]}-{row["q"]}'
                , axis=1)

In [18]:
space

,models,p,o,q,dist,llik
4,GARCH,1,0,1,Normal,-8077.079067
5,GARCH,1,0,1,StudentsT,-8092.003581
6,GARCH,1,0,2,Normal,-8077.068193
7,GARCH,1,0,2,StudentsT,-8087.546222
12,GARCH,2,0,1,Normal,-8077.135315
13,GARCH,2,0,1,StudentsT,-8094.279930
14,GARCH,2,0,2,Normal,-8076.707094
15,GARCH,2,0,2,StudentsT,-8093.429218
16,EGARCH,1,1,1,Normal,-8056.810828
17,EGARCH,1,1,1,StudentsT,-8084.812368


In [47]:
best_spec

,models,dist,llik,p,o,q
0,EGARCH,Normal,-8039.873169,2,1,1
1,EGARCH,StudentsT,-8082.723301,2,1,1
2,GARCH,Normal,-8076.707094,2,0,2
3,GARCH,StudentsT,-8087.546222,1,0,2


### Merge Predictions of best specifications

In [20]:
predictions_df = pd.DataFrame(index = eval_data_garch.index)
for mod in best_spec_strings : 
    predictions_df = predictions_df.merge(
        predictions[mod].add_suffix(f'_{mod}'),
        left_index=True,
        right_index = True, 
    )

In [21]:
# since demeaning drop empty columns with mu
drop_cols = predictions_df.columns[predictions_df.columns.map(lambda x: x.startswith('mean'))]
predictions_df = predictions_df.drop(drop_cols, axis=1)

### Add Descriptive Features Calculated from returns

In [22]:
desc_obj = DescStat(data=eval_data_garch['return100'])

In [80]:
desc_data = (
    pd.DataFrame(
        {
            'mean5_return100': desc_obj.calc_mean(window=5, fill_initial='first_constant'),
            'var5_return100': desc_obj.calc_var(window=5, fill_initial='first_constant'),
            'mean10_return100': desc_obj.calc_mean(window=10, fill_initial='first_constant'),
            'var10_return100': desc_obj.calc_var(window=10, fill_initial='first_constant'),
            'mean20_return100': desc_obj.calc_mean(window=20, fill_initial='first_constant'),
            'var20_return100': desc_obj.calc_var(window=20, fill_initial='first_constant'),
            'dt': eval_data_garch.index
        }
    ).set_index('dt')
)

In [81]:
# corr = desc_data.corr()
# corr_style = corr.style.background_gradient(cmap='coolwarm').set_precision(4)
# corr_style.to_excel('plots/corr_mat.xlsx')
desc_data.corr().style.background_gradient(cmap='coolwarm').set_precision(4)

,mean20_return100,var20_return100
mean20_return100,1.0000,-0.0244
var20_return100,-0.0244,1.0000


### Data for Neural Networks are prepared

In [82]:
enriched_ret_data = (
    eval_data_garch
        .merge(
            desc_data,
            left_index=True,
            right_index=True
        )
        .merge(
            predictions_df,
            left_index=True,
            right_index=True
        )
)

In [27]:
enriched_ret_data

,return100,mean5_return100,var5_return100,mean10_return100,var10_return100,mean20_return100,var20_return100,var_pred_EGARCH-Normal-2-1,var_pred_EGARCH-StudentsT-2-1,var_pred_GARCH-Normal-2-2,var_pred_GARCH-StudentsT-1-2
dt,,,,,,,,,,,
2001-05-31,0.917147,0.417483,1.022965,-0.180891,0.930303,0.095766,3.079633,4.425344,4.674149,4.232313,4.149881
2001-06-01,1.064754,0.417483,1.022965,-0.180891,0.930303,0.095766,3.079633,4.600829,4.417485,4.055842,3.990289
2001-06-04,0.918222,0.417483,1.022965,-0.180891,0.930303,0.095766,3.079633,4.172359,4.020876,3.953583,3.876728
2001-06-05,0.546293,0.417483,1.022965,-0.180891,0.930303,0.095766,3.079633,4.639103,3.698352,3.809992,3.736206
2001-06-06,-1.359004,0.417483,1.022965,-0.180891,0.930303,0.095766,3.079633,3.510432,3.438154,3.615520,3.530038
...,...,...,...,...,...,...,...,...,...,...,...
2014-02-06,2.520267,-0.427166,2.279567,-0.969217,2.951716,-0.696195,2.954110,3.741007,3.811220,3.029951,2.914756
2014-02-07,2.250651,-0.012942,4.044949,-0.488988,3.856921,-0.627900,3.313465,3.771760,3.685732,3.414492,3.239957
2014-02-10,-0.048953,0.799119,3.694561,0.010972,3.845653,-0.550524,3.650026,3.692544,3.660634,3.503424,3.376057


In [44]:
best_spec_strings

0       EGARCH-Normal-2-1
1    EGARCH-StudentsT-2-1
2        GARCH-Normal-2-2
3     GARCH-StudentsT-1-2
dtype: object

### Standardize the data

In [83]:
# initiate standardizer 
Standardizer_train = Standardizer(
    demean=True,
    rescale=True,
    skipna=True
)

In [84]:
# standardize all but first column where the returns are
train_data_standard = (
    enriched_ret_data
        .drop('return100', axis=1)
        .apply(
            func=Standardizer_train.perform_calc,
            axis=0,
            result_type='broadcast'
        )
        .join(enriched_ret_data[['return100']])
)

# Content of ANN_core - for easier manipulation here and then at the end of session copied to separate .py file

### ANN Estimation

In [92]:
NN_obj = NeuralNetwork(
        x=train_data_standard.drop('return100', axis=1).iloc[0:train_part,:],
        y=train_data_standard[['return100']].iloc[0:train_part],
#         x=train_data_standard.drop('return100', axis=1).iloc[train_part:(train_part+validation_part),:],
#         y=train_data_standard[['return100']].iloc[train_part:(train_part+validation_part)],
        obj_fun=lLik,
        dist='normal',
        hidden_layer=2,
        node_fun=sigmoid,
        num_nodes=[8,8],
        step_rate=0.0001,
        bias=True,
        L2_reg=False
)
    
iter = 10000
cost_train = np.zeros(iter)
cost_eval = np.zeros(iter)
NN_obj.feedforward()
for j in range(iter):

        NN_obj.backprop()
        NN_obj.feedforward()
        print(j)
        cost_train[j] = NN_obj.cost()[0]
        cost_eval[j] = NN_obj.cost(
            new_input = train_data_standard.drop('return100', axis=1).iloc[train_part:(train_part+validation_part),:], 
            new_y = train_data_standard[['return100']].iloc[train_part:(train_part+validation_part)]
        #             new_input = train_data_standard.drop('return100', axis=1).iloc[0:train_part,:], 
        #             new_y = train_data_standard[['return100']].iloc[0:train_part]            
        )[0]
        print(cost_train[j])
        print(cost_eval[j])
        if (j > 0): 
            if (cost_eval[j] < cost_eval[j-1]):
                break
        

0
-6782.51558512498
-2191.14345302928
1
-6681.337100235206
-2186.7146161254823
2
-6600.7336804427205
-2184.262417870967
3
-6534.902064806392
-2183.1024532412603
4
-6480.069199165466
-2182.812989993604
5
-6433.664500798344
-2183.121958596443


### Testing Dataset

##### ARCH

In [50]:
test_eval_merged_data = pd.concat([eval_data_garch, test_data])

In [51]:
ArchMod_test = ArchModelWrapper(train_data=test_eval_merged_data, eval_data_garch=test_data)

In [52]:
time_begin = perf_counter()
space['llik'] = best_spec.apply(lambda row: ArchMod_test.estimate_predict(row['models'], row['p'], row['o'], row['q'], row['dist']),
                                    axis=1)
f'Time of calculations: {round((perf_counter()-time_begin)/60, 0)} min'

'Time of calculations: 9.0 min'

In [53]:
predictions = ArchMod_test.get_predictions()

##### prepare data for NN

In [54]:
predictions_df_test = pd.DataFrame(index = test_eval_merged_data.index)
for mod in best_spec_strings : 
    predictions_df_test = predictions_df_test.merge(
        predictions[mod].add_suffix(f'_{mod}'),
        left_index=True,
        right_index = True, 
    )

In [55]:
# since demeaning drop empty columns with mu
drop_cols = predictions_df_test.columns[predictions_df_test.columns.map(lambda x: x.startswith('mean'))]
predictions_df_test = predictions_df_test.drop(drop_cols, axis=1)

In [93]:
desc_obj_test = DescStat(data=test_data['return100'])

In [94]:
desc_data_test = (
    pd.DataFrame(
        {
            'mean5_return100': desc_obj_test.calc_mean(window=5, fill_initial='first_constant'),
            'var5_return100': desc_obj_test.calc_var(window=5, fill_initial='first_constant'),
            'mean10_return100': desc_obj_test.calc_mean(window=10, fill_initial='first_constant'),
            'var10_return100': desc_obj_test.calc_var(window=10, fill_initial='first_constant'),
            'mean20_return100': desc_obj_test.calc_mean(window=20, fill_initial='first_constant'),
            'var20_return100': desc_obj_test.calc_var(window=20, fill_initial='first_constant'),
            'dt': test_data.index
        }
    ).set_index('dt')
)

In [95]:
enriched_ret_data_test = (
    test_data
        .merge(
            desc_data_test,
            left_index=True,
            right_index=True
        )
        .merge(
            predictions_df_test,
            left_index=True,
            right_index=True
        )
)

In [97]:
# standardize all but first column where the returns are
test_data_standard = (
    enriched_ret_data_test
        .drop('return100', axis=1)
        .apply(
            func=Standardizer_train.predict,
            axis=0,
            result_type='broadcast'
        )
        .join(test_data[['return100']])
)

In [98]:
nn_predictions_no_calc =  NN_obj.predict(new_input = test_data_standard.drop(['return100'], axis=1)) 

##### ANN

In [100]:
NN_obj_backup = NN_obj

In [104]:
NN_obj = NN_obj_backup

In [105]:
nn_predictions = np.array([])
counter = 0
for i in test_data_standard.index:
    counter = counter + 1 
    # new row
    new_row = test_data_standard.loc[[i],:].drop(['return100'], axis=1)
    # predict on it
    nn_predictions = np.append(nn_predictions, NN_obj.predict(new_input = new_row)) 
    
    print(NN_obj.input.shape)
    print(NN_obj.y.shape)
    
    NN_obj.add_data(
            new_x=test_data_standard.loc[[i],:].drop(['return100'], axis=1),
            new_y=test_data_standard.loc[[i],:][['return100']],
    )
    NN_obj.feedforward()
    
    if counter % 200 == 0:
        # recalculate after every week
        for j in range(100):
            print(i,j)
            NN_obj.backprop()
            NN_obj.feedforward()
            print(NN_obj.cost()[0])


(3758, 6)
(3758, 1)
(3759, 6)
(3759, 1)
(3760, 6)
(3760, 1)
(3761, 6)
(3761, 1)
(3762, 6)
(3762, 1)
(3763, 6)
(3763, 1)
(3764, 6)
(3764, 1)
(3765, 6)
(3765, 1)
(3766, 6)
(3766, 1)
(3767, 6)
(3767, 1)
(3768, 6)
(3768, 1)
(3769, 6)
(3769, 1)
(3770, 6)
(3770, 1)
(3771, 6)
(3771, 1)
(3772, 6)
(3772, 1)
(3773, 6)
(3773, 1)
(3774, 6)
(3774, 1)
(3775, 6)
(3775, 1)
(3776, 6)
(3776, 1)
(3777, 6)
(3777, 1)
(3778, 6)
(3778, 1)
(3779, 6)
(3779, 1)
(3780, 6)
(3780, 1)
(3781, 6)
(3781, 1)
(3782, 6)
(3782, 1)
(3783, 6)
(3783, 1)
(3784, 6)
(3784, 1)
(3785, 6)
(3785, 1)
(3786, 6)
(3786, 1)
(3787, 6)
(3787, 1)
(3788, 6)
(3788, 1)
(3789, 6)
(3789, 1)
(3790, 6)
(3790, 1)
(3791, 6)
(3791, 1)
(3792, 6)
(3792, 1)
(3793, 6)
(3793, 1)
(3794, 6)
(3794, 1)
(3795, 6)
(3795, 1)
(3796, 6)
(3796, 1)
(3797, 6)
(3797, 1)
(3798, 6)
(3798, 1)
(3799, 6)
(3799, 1)
(3800, 6)
(3800, 1)
(3801, 6)
(3801, 1)
(3802, 6)
(3802, 1)
(3803, 6)
(3803, 1)
(3804, 6)
(3804, 1)
(3805, 6)
(3805, 1)
(3806, 6)
(3806, 1)
(3807, 6)
(3807, 1)


(3965, 6)
(3965, 1)
(3966, 6)
(3966, 1)
(3967, 6)
(3967, 1)
(3968, 6)
(3968, 1)
(3969, 6)
(3969, 1)
(3970, 6)
(3970, 1)
(3971, 6)
(3971, 1)
(3972, 6)
(3972, 1)
(3973, 6)
(3973, 1)
(3974, 6)
(3974, 1)
(3975, 6)
(3975, 1)
(3976, 6)
(3976, 1)
(3977, 6)
(3977, 1)
(3978, 6)
(3978, 1)
(3979, 6)
(3979, 1)
(3980, 6)
(3980, 1)
(3981, 6)
(3981, 1)
(3982, 6)
(3982, 1)
(3983, 6)
(3983, 1)
(3984, 6)
(3984, 1)
(3985, 6)
(3985, 1)
(3986, 6)
(3986, 1)
(3987, 6)
(3987, 1)
(3988, 6)
(3988, 1)
(3989, 6)
(3989, 1)
(3990, 6)
(3990, 1)
(3991, 6)
(3991, 1)
(3992, 6)
(3992, 1)
(3993, 6)
(3993, 1)
(3994, 6)
(3994, 1)
(3995, 6)
(3995, 1)
(3996, 6)
(3996, 1)
(3997, 6)
(3997, 1)
(3998, 6)
(3998, 1)
(3999, 6)
(3999, 1)
(4000, 6)
(4000, 1)
(4001, 6)
(4001, 1)
(4002, 6)
(4002, 1)
(4003, 6)
(4003, 1)
(4004, 6)
(4004, 1)
(4005, 6)
(4005, 1)
(4006, 6)
(4006, 1)
(4007, 6)
(4007, 1)
(4008, 6)
(4008, 1)
(4009, 6)
(4009, 1)
(4010, 6)
(4010, 1)
(4011, 6)
(4011, 1)
(4012, 6)
(4012, 1)
(4013, 6)
(4013, 1)
(4014, 6)
(4014, 1)


(4392, 6)
(4392, 1)
(4393, 6)
(4393, 1)
(4394, 6)
(4394, 1)
(4395, 6)
(4395, 1)
(4396, 6)
(4396, 1)
(4397, 6)
(4397, 1)
(4398, 6)
(4398, 1)
(4399, 6)
(4399, 1)
(4400, 6)
(4400, 1)
(4401, 6)
(4401, 1)
(4402, 6)
(4402, 1)
(4403, 6)
(4403, 1)
(4404, 6)
(4404, 1)
(4405, 6)
(4405, 1)
(4406, 6)
(4406, 1)
(4407, 6)
(4407, 1)
(4408, 6)
(4408, 1)
(4409, 6)
(4409, 1)
(4410, 6)
(4410, 1)
(4411, 6)
(4411, 1)
(4412, 6)
(4412, 1)
(4413, 6)
(4413, 1)
(4414, 6)
(4414, 1)
(4415, 6)
(4415, 1)
(4416, 6)
(4416, 1)
(4417, 6)
(4417, 1)
(4418, 6)
(4418, 1)
(4419, 6)
(4419, 1)
(4420, 6)
(4420, 1)
(4421, 6)
(4421, 1)
(4422, 6)
(4422, 1)
(4423, 6)
(4423, 1)
(4424, 6)
(4424, 1)
(4425, 6)
(4425, 1)
(4426, 6)
(4426, 1)
(4427, 6)
(4427, 1)
(4428, 6)
(4428, 1)
(4429, 6)
(4429, 1)
(4430, 6)
(4430, 1)
(4431, 6)
(4431, 1)
(4432, 6)
(4432, 1)
(4433, 6)
(4433, 1)
(4434, 6)
(4434, 1)
(4435, 6)
(4435, 1)
(4436, 6)
(4436, 1)
(4437, 6)
(4437, 1)
(4438, 6)
(4438, 1)
(4439, 6)
(4439, 1)
(4440, 6)
(4440, 1)
(4441, 6)
(4441, 1)


(4599, 6)
(4599, 1)
(4600, 6)
(4600, 1)
(4601, 6)
(4601, 1)
(4602, 6)
(4602, 1)
(4603, 6)
(4603, 1)
(4604, 6)
(4604, 1)
(4605, 6)
(4605, 1)
(4606, 6)
(4606, 1)
(4607, 6)
(4607, 1)
(4608, 6)
(4608, 1)
(4609, 6)
(4609, 1)
(4610, 6)
(4610, 1)
(4611, 6)
(4611, 1)
(4612, 6)
(4612, 1)
(4613, 6)
(4613, 1)
(4614, 6)
(4614, 1)
(4615, 6)
(4615, 1)
(4616, 6)
(4616, 1)
(4617, 6)
(4617, 1)
(4618, 6)
(4618, 1)
(4619, 6)
(4619, 1)
(4620, 6)
(4620, 1)
(4621, 6)
(4621, 1)
(4622, 6)
(4622, 1)
(4623, 6)
(4623, 1)
(4624, 6)
(4624, 1)
(4625, 6)
(4625, 1)
(4626, 6)
(4626, 1)
(4627, 6)
(4627, 1)
(4628, 6)
(4628, 1)
(4629, 6)
(4629, 1)
(4630, 6)
(4630, 1)
(4631, 6)
(4631, 1)
(4632, 6)
(4632, 1)
(4633, 6)
(4633, 1)
(4634, 6)
(4634, 1)
(4635, 6)
(4635, 1)
(4636, 6)
(4636, 1)
(4637, 6)
(4637, 1)
(4638, 6)
(4638, 1)
(4639, 6)
(4639, 1)
(4640, 6)
(4640, 1)
(4641, 6)
(4641, 1)
(4642, 6)
(4642, 1)
(4643, 6)
(4643, 1)
(4644, 6)
(4644, 1)
(4645, 6)
(4645, 1)
(4646, 6)
(4646, 1)
(4647, 6)
(4647, 1)
(4648, 6)
(4648, 1)


(5016, 6)
(5016, 1)
(5017, 6)
(5017, 1)
(5018, 6)
(5018, 1)
(5019, 6)
(5019, 1)
(5020, 6)
(5020, 1)
(5021, 6)
(5021, 1)
(5022, 6)
(5022, 1)
(5023, 6)
(5023, 1)
(5024, 6)
(5024, 1)
(5025, 6)
(5025, 1)
(5026, 6)
(5026, 1)
(5027, 6)
(5027, 1)
(5028, 6)
(5028, 1)
(5029, 6)
(5029, 1)
(5030, 6)
(5030, 1)
(5031, 6)
(5031, 1)
(5032, 6)
(5032, 1)
(5033, 6)
(5033, 1)
(5034, 6)
(5034, 1)
(5035, 6)
(5035, 1)
(5036, 6)
(5036, 1)
(5037, 6)
(5037, 1)
(5038, 6)
(5038, 1)
(5039, 6)
(5039, 1)
(5040, 6)
(5040, 1)
(5041, 6)
(5041, 1)
(5042, 6)
(5042, 1)
(5043, 6)
(5043, 1)
(5044, 6)
(5044, 1)
(5045, 6)
(5045, 1)
(5046, 6)
(5046, 1)
(5047, 6)
(5047, 1)
(5048, 6)
(5048, 1)
(5049, 6)
(5049, 1)
(5050, 6)
(5050, 1)
(5051, 6)
(5051, 1)
(5052, 6)
(5052, 1)
(5053, 6)
(5053, 1)
(5054, 6)
(5054, 1)
(5055, 6)
(5055, 1)
(5056, 6)
(5056, 1)
(5057, 6)
(5057, 1)
(5058, 6)
(5058, 1)
(5059, 6)
(5059, 1)
(5060, 6)
(5060, 1)
(5061, 6)
(5061, 1)
(5062, 6)
(5062, 1)
(5063, 6)
(5063, 1)
(5064, 6)
(5064, 1)
(5065, 6)
(5065, 1)


(5217, 6)
(5217, 1)
(5218, 6)
(5218, 1)
(5219, 6)
(5219, 1)
(5220, 6)
(5220, 1)
(5221, 6)
(5221, 1)
(5222, 6)
(5222, 1)
(5223, 6)
(5223, 1)
(5224, 6)
(5224, 1)
(5225, 6)
(5225, 1)
(5226, 6)
(5226, 1)
(5227, 6)
(5227, 1)
(5228, 6)
(5228, 1)
(5229, 6)
(5229, 1)
(5230, 6)
(5230, 1)
(5231, 6)
(5231, 1)
(5232, 6)
(5232, 1)
(5233, 6)
(5233, 1)
(5234, 6)
(5234, 1)
(5235, 6)
(5235, 1)
(5236, 6)
(5236, 1)
(5237, 6)
(5237, 1)
(5238, 6)
(5238, 1)
(5239, 6)
(5239, 1)
(5240, 6)
(5240, 1)
(5241, 6)
(5241, 1)
(5242, 6)
(5242, 1)
(5243, 6)
(5243, 1)
(5244, 6)
(5244, 1)
(5245, 6)
(5245, 1)
(5246, 6)
(5246, 1)
(5247, 6)
(5247, 1)
(5248, 6)
(5248, 1)
(5249, 6)
(5249, 1)
(5250, 6)
(5250, 1)
(5251, 6)
(5251, 1)
(5252, 6)
(5252, 1)
(5253, 6)
(5253, 1)
(5254, 6)
(5254, 1)
(5255, 6)
(5255, 1)
(5256, 6)
(5256, 1)
(5257, 6)
(5257, 1)
(5258, 6)
(5258, 1)
(5259, 6)
(5259, 1)
(5260, 6)
(5260, 1)
(5261, 6)
(5261, 1)
(5262, 6)
(5262, 1)
(5263, 6)
(5263, 1)
(5264, 6)
(5264, 1)
(5265, 6)
(5265, 1)
(5266, 6)
(5266, 1)


In [106]:
nn_predictions_df = pd.DataFrame({'pred': nn_predictions}, index = test_data_standard.index)

##### Variable importance

In [1]:
for var in input_data_nn.columns :
    
    print(var)
    llik_change = np.array([])
    
    for j in range(10):
        input_data_nn_temp = input_data_nn
        input_data_nn_temp[var] = np.random.permutation(input_data_nn_temp[var].values)
        
        llik_change = np.append(llik_change, NN_obj.cost(new_input = input_data_nn_temp, new_y = NN_obj.y)[0])
    print(llik_change)
    avg_llik_change = llik_change.mean()
    avg_change.loc[avg_change['variable'] == var, 'llik_change_avg'] = avg_llik_change

NameError: name 'input_data_nn' is not defined

In [73]:
avg_change

,variable,llik_change_avg
0,mean5_return100,-17982.3
1,var5_return100,-18007
2,mean10_return100,-18053.2
3,var10_return100,-18119.1
4,mean20_return100,-18140.9
5,var20_return100,-18147.2
6,var_pred_EGARCH-Normal-1-1,-18457.4
7,var_pred_EGARCH-StudentsT-2-2,-18484.5
8,var_pred_GARCH-Normal-2-1,-18470.2
9,var_pred_GARCH-StudentsT-1-1,-18808.3
